In [1]:
import pandas as pd


Very basic idea here is to create a network of democrats / liberals and segment them for the upcoming primaries (i.e. centrists, leftists, etc.)

In [2]:
import tweepy, sys, os, logging, json
from twitter_config import consumer_key, consumer_secret

In [3]:
logging.basicConfig(filename='twitter_error_log_rule_change.log',filemode='w', level=logging.ERROR)

First I'm just going to store JSON log files, but eventually I'm going to try to load it into postgres, because I think that makes the most since. I am using the code from https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./

In [4]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
 
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True,
                 parser=tweepy.parsers.JSONParser()
                )
 
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)

In [14]:
# Here are my search terms - I can change the text string if I want to change my terms
# topic_search = ('aoc OR #aoc OR (alexandria AND ocasio-cortez) OR (alexandria AND ocasio) OR (alexandria AND cortez) OR ocasio-cortez OR ocasiocortez OR (alexandria AND ocasio AND cortez)')
# topic_search = ('(captain AND marvel) OR captainmarvel OR #captainmarvel OR (brie AND larson) OR brielarson OR #brielarson')
# topic_search = ('#QAnon OR #Qarmy OR QAnon OR Qarmy OR #WWG1WGA OR WWG1WGA OR #TrustThePlan OR #GreatAwakening')
topic_search = ('(3 AND batters) OR (three AND batters) OR (rules AND changes AND (baseball OR MLB)) OR (rule AND change AND (baseball OR MLB)) OR (rob AND manfred) OR (batter AND minimum)')


searchQuery = topic_search

## Download as many search-relevant tweets from the API as possible, while respecting the rate limit.
This code is largely taken from https://bhaskarvk.github.io/2015/01/how-to-use-twitters-search-rest-api-most-effectively./
tweaked it to save json files every 4000 tweets, and to use logging rather than printing.

ALso, adding tweet_mode = extended to every api.search method, and changing langauge as well

In [15]:
maxTweets = 2000000000 # Some arbitrary large number
tweetsPerQry = 100  # this is the max the API permits

# If results from a specific ID onwards are reqd, set since_id to that ID.
# else default to no lower limit, go as far back as API allows
sinceId = None

# If results only below a specific ID are reqd, set max_id to that ID.
# else default to no upper limit, start from the most recent tweet matching the search query.
max_id = -1

tweetCount = 0
tweetFiles = 0
msg =  "Downloading max {0} tweets".format(maxTweets)
logging.error(msg)
tweets = []
while 4000*tweetFiles + tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en')
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        since_id=sinceId)
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        max_id=str(max_id - 1))
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode='extended', lang='en',
                                        max_id=str(max_id - 1),
                                        since_id=sinceId)
                
        new_tweets = new_tweets['statuses']
        if len(new_tweets) == 0:
            msg = 'No tweets found'
            logging.error(msg)
            break
        
        tweets.extend(new_tweets)
        tweetCount += len(new_tweets)
        msg = "Downloaded {0} tweets".format(tweetCount + tweetFiles*4000)
        logging.error(msg)
        max_id = new_tweets[-1]['id']
        
        if tweetCount > 4000:
            with open('tweets'+str(tweetFiles)+'.json', 'w', encoding="utf8") as outfile:  
                json.dump(tweets, outfile)
            msg = "JSON file saved"
            logging.error(msg)
            tweetFiles += 1
            tweetCount = 0
            del tweets[:]
                
    except tweepy.TweepError as e:
        msg = 'Query failed when max_id equaled {0}: {1}'.format(max_id, e)
        logging.error(msg)

if tweetCount > 0:
    with open('tweets'+str(tweetFiles)+'.json', 'w', encoding="utf8") as outfile:  
        json.dump(tweets, outfile)
    msg = "JSON file saved"
    logging.error(msg)
    tweetFiles += 1
    tweetCount = 0

In [29]:
print(max_id)

1106243585958182912


In [7]:
with open('tweets0.json') as json_file:  
    test = json.load(json_file)

In [8]:
len(test)

90

In [13]:
test[89]

{'created_at': 'Sat Mar 09 14:58:21 +0000 2019',
 'id': 1104396007440146432,
 'id_str': '1104396007440146432',
 'full_text': '@SportsNetLA 3 batter minimum \n\nRobot umpire takes something away from the art of pitch framing.\n\nShifts are good for baseball, though batters aren’t doing it yet, it forces batters to hit to the opposite field or bunt.\n\nMoving mound is going to cause injuries from trying to compensate',
 'truncated': False,
 'display_text_range': [13, 286],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'SportsNetLA',
    'name': 'SportsNet LA',
    'id': 2233347469,
    'id_str': '2233347469',
    'indices': [0, 12]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': 1104150328604155904,
 'in_reply_to_status_id_str': '1104150328604155904',
 'in_reply_to_user_id': 2233347469,
 'in_re

In [30]:
with open('tweets107.json') as json_file:  
    test_last = json.load(json_file)

In [31]:
test_tweet = test_last[50]


In [32]:
test_tweet['retweeted_status']['full_text']

KeyError: 'retweeted_status'

In [27]:
test_tweet

{'created_at': 'Thu Mar 14 19:19:14 +0000 2019',
 'id': 1106273600821035013,
 'id_str': '1106273600821035013',
 'full_text': 'RT @outlawjw: Sorry, but all the (one-way) tickets have ALREADY been accounted for.  And this cruise is all set to get underway.\n\n#GreatAwa…',
 'truncated': False,
 'display_text_range': [0, 140],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': 'outlawjw',
    'name': 'OutlawJoseyWales⚒️',
    'id': 25866103,
    'id_str': '25866103',
    'indices': [3, 12]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 1043688422844456961,
  'id_str': '1043688422844456961',
  'name': 'Gemmyguy7',
  'screen_name': 'billybassguy',
 

So now I have the barebones code - I'll create a new notebook where I make functions to select and clean the twitter data I need on just this one dataframe, and then I'll apply it to all of my dataframes and combine them together